In [ ]:
!pip install qiskit
!pip install qiskit_aer
import numpy as np
import random
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
import matplotlib.pyplot as plt

# Import noise modeling tools
from qiskit_aer.noise import NoiseModel, depolarizing_error

# --- 1. Constants ---
BASES = ['Z', 'X']  # Z-basis |0>, |1> and X-basis |+>, |->
QBER_THRESHOLD = 0.10  # 10% error threshold to detect Eve

# --- 2. Helper Functions ---

def get_noise_model(noise_prob):
    """Creates a simple depolarizing noise model."""
    noise_model = NoiseModel()

    # Add depolarizing error to single-qubit gates (H and X)
    error = depolarizing_error(noise_prob, 1)
    noise_model.add_quantum_error(error, ['h', 'x'], [0])

    # Add depolarizing error to measurement
    error_meas = depolarizing_error(noise_prob, 1)
    noise_model.add_quantum_error(error_meas, "measure", [0])

    return noise_model

def encode_qubit(bit, basis):
    """Creates a quantum circuit to encode a single bit in a given basis."""
    qc = QuantumCircuit(1)

    if basis == 'Z':
        if bit == 1:
            qc.x(0)  # |1>
        # if bit == 0, do nothing (already |0>)
    elif basis == 'X':
        if bit == 0:
            qc.h(0)  # |+>
        else:
            qc.x(0)
            qc.h(0)  # |->

    qc.barrier()
    return qc

def simulate_channel_with_eve(alice_circuit, simulator, eve_active=False):
    """
    Simulates the quantum channel. If Eve is active, she performs an
    intercept-resend attack using the *same* simulator (which may be noisy).
    """
    if not eve_active:
        # No Eve, channel is "perfect" (aside from any environmental noise)
        return alice_circuit

    # Eve is active: Intercept-Resend Attack
    eve_basis = random.choice(BASES)

    # Eve prepares to measure in her basis
    qc_eve = alice_circuit.copy()
    if eve_basis == 'X':
        qc_eve.h(0)
    qc_eve.measure_all()

    # Eve runs her measurement
    result = simulator.run(transpile(qc_eve, simulator), shots=1).result()
    counts = result.get_counts(qc_eve)
    eve_bit = int(list(counts.keys())[0])

    # Eve creates a NEW qubit and sends it to Bob
    qc_resend = encode_qubit(eve_bit, eve_basis)

    return qc_resend

# --- 3. Main BB84 Simulation ---

def run_bb84(key_length=1000, eve_active=False, noise_prob=0.0):
    """
    Runs a full BB84 simulation, optimized to batch Bob's measurements.
    Can simulate both an eavesdropper and environmental noise.
    """

    # 1. Setup Simulator (with or without noise)
    if noise_prob > 0:
        noise_model = get_noise_model(noise_prob)
        simulator = AerSimulator(noise_model=noise_model)
        print(f"\n--- BB84 Sim (Eve: {eve_active}, Noise: {noise_prob*100}%) ---")
    else:
        simulator = AerSimulator()
        print(f"\n--- BB84 Sim (Eve: {eve_active}, Noise: None) ---")

    # 2. Alice generates her bits and bases
    alice_bits = np.random.randint(2, size=key_length)
    alice_bases = np.random.choice(BASES, size=key_length)
    bob_bases = np.random.choice(BASES, size=key_length)

    circuits_to_batch = []

    for bit, a_basis, b_basis in zip(alice_bits, alice_bases, bob_bases):

        # 3. Alice encodes her qubit
        qc_alice = encode_qubit(bit, a_basis)

        # 4. Qubit goes through the channel (with/without Eve)
        qc_incoming = simulate_channel_with_eve(qc_alice, simulator, eve_active)

        # 5. Bob prepares his measurement circuit
        qc_final = qc_incoming.copy()
        if b_basis == 'X':
            qc_final.h(0)  # Apply H-gate for X-basis measurement
        qc_final.measure_all()

        circuits_to_batch.append(qc_final)

    if not circuits_to_batch:
        return {'QBER': 1.0, 'Key_Length': 0}

    # 6. Bob runs ALL his circuits in one batch
    # print(f"[Executing Batch] Running {len(circuits_to_batch)} circuits...")
    transpiled_circuits = transpile(circuits_to_batch, simulator)
    result = simulator.run(transpiled_circuits, shots=1).result()

    bob_bits_measured = np.array([
        int(list(result.get_counts(qc).keys())[0])
        for qc in circuits_to_batch
    ])

    # 7. Sifting: Find indices where bases matched
    matching_indices = np.where(alice_bases == bob_bases)[0]

    if len(matching_indices) == 0:
        return {'QBER': 1.0, 'Key_Length': 0}

    # 8. Get the "sifted" keys
    raw_key_alice = alice_bits[matching_indices]
    raw_key_bob = bob_bits_measured[matching_indices]
    key_length_sifted = len(raw_key_alice)

    # 9. Error Calculation (QBER)
    errors = np.sum(raw_key_alice != raw_key_bob)
    qber = errors / key_length_sifted

    # 10. Security Check
    is_secure = qber < QBER_THRESHOLD

    print(f"Sifted Key Length: {key_length_sifted}")
    print(f"Errors found:      {errors}")
    print(f"QBER:              {qber:.4f}")
    print(f"Key is considered {'SECURE' if is_secure else 'INSECURE (Eve detected!)'}.")

    return {
        'QBER': qber,
        'Key_Length': key_length_sifted
    }

# --- 4. Main Execution & Plotting ---

if __name__ == "__main__":

    print("\n\n=== BB84 PROTOCOL ANALYSIS ===")
    print("="*30)

    # --- Part 1: Run simulations for Eve vs. No Eve ---
    print("\n[Running Scenario: Secure Channel]")
    result_secure = run_bb84(key_length=2000, eve_active=False, noise_prob=0.0)

    print("\n[Running Scenario: Eavesdropper Active]")
    result_insecure = run_bb84(key_length=2000, eve_active=True, noise_prob=0.0)

    # --- Part 2: Run simulations for Noise Impact ---
    print("\n[Running Scenario: Environmental Noise Impact]")
    noise_levels = np.linspace(0, 0.15, 10) # 0% to 15% noise
    qber_results = []

    for prob in noise_levels:
        # Run a smaller sim inside the loop so it's fast
        result = run_bb84(key_length=500, eve_active=False, noise_prob=prob)
        qber_results.append(result['QBER'])

    # --- Part 3: Plot Key Metrics (as required by prompt) ---
    print("\n[Generating Plots...]")
    plt.figure(figsize=(15, 6))

    # Plot 1: Eve Detection
    plt.subplot(1, 2, 1)
    bar_labels = ['Secure (No Eve)', 'Insecure (With Eve)']
    qber_values = [result_secure['QBER'], result_insecure['QBER']]
    colors = ['blue', 'red']

    plt.bar(bar_labels, qber_values, color=colors)
    plt.title('Eavesdropper (Eve) Detection', fontsize=14)
    plt.ylabel('Quantum Bit Error Rate (QBER)', fontsize=12)
    plt.axhline(y=QBER_THRESHOLD, color='gray', linestyle='--', label=f'Security Threshold ({QBER_THRESHOLD*100}%)')
    plt.legend()
    # Add text for the expected QBER
    plt.text(1, result_insecure['QBER'] - 0.05, f'Expected QBER ~0.25', ha='center', color='white', weight='bold')

    # Plot 2: Noise Impact
    plt.subplot(1, 2, 2)
    plt.plot(noise_levels * 100, qber_results, 'bo-', label='Measured QBER')
    plt.title('Impact of Environmental Noise', fontsize=14)
    plt.xlabel('Depolarizing Noise Probability (%)', fontsize=12)
    plt.ylabel('Quantum Bit Error Rate (QBER)', fontsize=12)
    plt.axhline(y=QBER_THRESHOLD, color='red', linestyle='--', label=f'Security Threshold ({QBER_THRESHOLD*100}%)')
    plt.legend()
    plt.grid(True)

    plt.suptitle('BB84 Hackathon: Key Metrics Analysis', fontsize=16)
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])

    # Save the plot for the report
    plt.savefig('bb84_key_metrics.png')

    # Show the plot
    plt.show()

    print("\n--- Final Conclusion ---")
    print(f"Secure Run (No Noise) QBER: {result_secure['QBER']:.4f}")
    print(f"Insecure Run (Eve) QBER:    {result_insecure['QBER']:.4f}")
    print(f"The high QBER in the insecure run (expected ~0.25) confirms successful detection of the eavesdropper.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 104.4 MB/s eta 0:00:00


=== BB84 PROTOCOL ANALYSIS ===

[Running Scenario: Secure Channel]

--- BB84 Sim (Eve: False, Noise: None) ---
Sifted Key Length: 985
Errors found:      0
QBER:              0.0000
Key is considered SECURE.

[Running Scenario: Eavesdropper Active]

--- BB84 Sim (Eve: True, Noise: None) ---
